## Calculating Snow Depth and Ice Thickness Distributions

Snow depth and ice thickness can vary over length scales as short as 100 m.  However, grids of modelled snow depths and satellite-derived ice thicknesses are generally available with cell spacings of 25 km.  For snow depths this grid spacing is determined by the resolution of output grids from atmospheric reanalyses.  For satellite-derived ice thickness products, grid spacing is limited by either the extent of satellite footprints and/or the number of data points to generate grids from sparse points and transects.  It is advantageous to account for subgrid-scale variability of snow depths and ice thickness.  This notebook demonstrates how sub-grid-scale variability is accounted for in the radiative transfer model.

Mallet et al (2021) developed probability distributions for snow depth anomalies for snow on sea ice using snow data collected along transects at Russian drifting stations [refe].  Snow depths for each transect measured on a given date were standardized using the mean and standard deviation of snow depths for each transect.  Standard anomaly snow depths $d'$ for each transect are given by:

$$
d' = \frac{d_i - \bar{d_i}}{\sigma_{d_i}}
$$

where $d_i$ is a snow depth measured along transect $i$. 

![Linear relationship between standard deviations and mean snow depth for all data, and for winter and summer data (a,c); and distributions of snow depth anomalies (Mallet et al, 2021, figure 3.](https://github.com/robbiemallett/sub_km/blob/main/figures/fig_3.png)

A linear relationship between mean snow depth $\bar{d}$ and standard deviation of snow depths $\sigma_d$ for each transect is estimated.  The best fit line is given by

$$
\sigma_{d} = 0.417 \times \bar{d}
$$

The distribution of standardized anomalies can be approximated by a skew-normal distribution with skewness $a = 2.54$, location $\xi = -1.11$ and scale $\omega = 1.5$  

In [ ]:
import numpy as np
from scipy.stats import skewnorm

import matplotlib.pyplot as plt

from beer_lambert_rt.distributions import (snow_depth_std, snow_depth_distribution, 
                                           snow_depth_anomaly_distribution, ice_thickness_distribution)

The `scipy.stats.skewnorm` class can be used to define a continuous random variate for the Skew-norm distribution.

In [ ]:
rv = skewnorm(a=2.54, loc=-1.11, scale=1.5)

x = np.linspace(-5., 5., 100)
p = rv.pdf(x)

fig, ax = plt.subplots(figsize=(7,5))
ax.axvline(0., color='k', zorder=0)
ax.plot(x, p)
ax.set_xlabel('Snow Depth Anomaly (m)')
ax.set_ylabel('Probability Density')

Snow depth anomalies can be transformed into snow depths by multiplying by the standard deviation and adding the mean snow depth.  Where the standard deviation $\sigma_d$ is given by $0.417 \times \bar{d}$.

In [ ]:
# Add plot showing distribution of snow depth

# Make join ice thickness and snow depth distributions using mapgrid

In [ ]:
snow_depth_mean = 0.3
nbins = 7
factor = 3.
width = factor * snow_depth_mean / nbins
depth, prob = snow_depth_distribution(snow_depth_mean, nbins=nbins, factor=factor)
depth, prob, prob.sum()

The transformed snow depths are shown below for a mean snow depth 0.3 m.

In [ ]:
x = np.linspace(-4., 4., 50)
y = snow_depth_anomaly_distribution.pdf(x)

x = x * snow_depth_std(snow_depth_mean) + snow_depth_mean

fig, ax = plt.subplots(figsize=(7,5))
ax.axvline(0., color='k', zorder=0)

ax.plot(x, y)
ax.bar(depth, prob, width=width, color='y', alpha=0.5)

The snow depth distribution is a continuous density function.  A discrete distribution is needed for modelling, so the distribution needs to be quantized into N equal-width bins over some interval.

For snow data, we decide on 7 bins for the interval 0 to $2 \bar{h}$.  In this example, the width of each bin is $\frac{2 \bar{h}}{7}$.

_NB.  The implicit assumption here is that if $\bar{h}$ is greater than zero, the gridcell, transect or area of interest, is always covered by some snow._.

## Ice Thickness Distribution

In [ ]:
(ice_thickness_bins, ice_thickness_width), ice_thickness_prob = ice_thickness_distribution(1.5)

In [ ]:
ice_thickness_bins, ice_thickness_width, ice_thickness_prob

In [ ]:
fig, ax = plt.subplots(figsize=(7,5))

ax.set_xlim(0, 1.5)
ax.bar(ice_thickness_bins, ice_thickness_prob, width=ice_thickness_width)